This script will make it so all images are normalized the same by taking the global min and max per band and applying it to each image.  Essentially I need to find across all training data and training bands what is the minimum and maximum values so all images are normalized the same.  I do this to avoid tiling appearences when deploying the model 

In [7]:
import pandas as pd
import numpy as np
import os

Function to get the the min-max globally but use the 5 and 95% cutoffs, I am not using this right now. 

In [ ]:
def minMax(x):
    return pd.Series(index=['min','max'],data=[x.percentile(5),x.percentile(95)])


Apply the minMax function by looping through all data.  Note we have so much data I sample only 30% of it per image. 

In [8]:
#path to training, validation and testing, note these paths can change
train = get_files('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/l8_sent_collection2_0_training_data_512')
test = get_files('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/l8_sent_collection2_0_testing_data_512')
val = get_files('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/l8_sent_collection2_0_validation_data_512')


final = []


def get_min_max(pathways):
    
    
    for f in pathways:

        base = os.path.basename(f)

        x = np.load(f)
        x = x.astype(float)
        x[x == 0] = np.nan
        x[x == -999] = np.nan

        #convert to dataframe
        in_shape = x.shape

        #turn to dataframe to normalize
        x = x.reshape(x.shape[0] * x.shape[1], x.shape[2])
    
        #sample 30% per image
        x = pd.DataFrame(x).dropna().sample(frac = 0.3)
        final.append(x)
        
    return()

get_min_max(train)
get_min_max(test)
get_min_max(val)

()

In [11]:
#concat the training, testing and validation and save to a file.  Using 0.01 and 0.99 cutoffs. 
final2 = pd.concat(final).quantile([.01, .99])
final2
final2.reset_index().to_csv("/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/l8_sent_collection2_0_512_global_min_max_cutoff.csv", index = False)

,0,1,2,3,4,5,6,7,8
0.01,-62.0,-58.0,-61.0,-93.0,-116.0,-125.0,-238.0,-238.0,-212.0
0.99,49.0,51.0,51.0,159.0,91.0,54.0,881.0,527.0,610.0
